In [8]:
import sys
import warnings
import pandas as pd

sys.path.append('../src')
warnings.simplefilter('ignore')

from preprocessing import *
from sklearn.pipeline import Pipeline as sklearn_pipeline

In [ ]:
articles = pd.read_csv('../data/interim/articles_new.csv')

chars_map = {'\xad': ' ', '…': '...',
             '«': '', '»': '',
             '"': '', '\'': '',
             '’': '', '‘': '',
             '”': '', '“': '', '„': '',
             '`': '', '*': ''}

pipeline = sklearn_pipeline([('replace_chars', ReplaceChars(chars_map)),
                             ('sub_spaces', RegExprSub(r' +', ' ')), 
                             ('strip', Strip()),
                             ('sent_tokenize', RusSentTokenizer()),
                             ('spell', Yandex_Speller()),
                             ('word_tokenize', Spacy_RusWordTokenizer()),
                             ('space_detect', SpaceDetecter()),
                             ('morph_predict', MorphPredictor()),
                             ('conllu_encode', CoNLLUFormatEncoder()),
                             ('syntax_parse', SyntaxParser('../models/parser_model.udpipe'))])

print('Transform titles')
articles['preproc_title'] = pipeline.fit_transform(articles.title)

print('Transform texts')
articles['preproc_text'] = pipeline.fit_transform(articles.text)

articles.to_csv('../data/interim/articles_preproc.csv', index=False)